In [1]:
!ls ../input

gapdataset  gendered-pronoun-resolution


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gapdataset', 'gendered-pronoun-resolution']


In [3]:
gap_development = pd.read_csv('../input/gapdataset/gap-development.tsv', delimiter='\t')
gap_test = pd.read_csv('../input/gapdataset/gap-test.tsv', delimiter='\t')
gap_validation = pd.read_csv('../input/gapdataset/gap-validation.tsv', delimiter='\t')

In [4]:
gap_development.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [5]:
gap_test.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [6]:
gap_validation.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,validation-1,He admitted making four trips to China and pla...,him,256,Jose de Venecia Jr,208,False,Abalos,241,False,http://en.wikipedia.org/wiki/Commission_on_Ele...
1,validation-2,"Kathleen Nott was born in Camberwell, London. ...",She,185,Ellen,110,False,Kathleen,150,True,http://en.wikipedia.org/wiki/Kathleen_Nott
2,validation-3,"When she returns to her hotel room, a Liberian...",his,435,Jason Scott Lee,383,False,Danny,406,True,http://en.wikipedia.org/wiki/Hawaii_Five-0_(20...
3,validation-4,"On 19 March 2007, during a campaign appearance...",he,333,Reucassel,300,True,Debnam,325,False,http://en.wikipedia.org/wiki/Craig_Reucassel
4,validation-5,"By this time, Karen Blixen had separated from ...",she,427,Finch Hatton,290,False,Beryl Markham,328,True,http://en.wikipedia.org/wiki/Denys_Finch_Hatton


In [7]:
train = pd.concat((gap_test, gap_validation)).reset_index(drop=True)
test = gap_development

train['A-coref'] = train['A-coref'].astype(int)
train['B-coref'] = train['B-coref'].astype(int)
train['NEITHER'] = 1.0 - (train['A-coref'] + train['B-coref'])

test['A-coref'] = test['A-coref'].astype(int)
test['B-coref'] = test['B-coref'].astype(int)
test['NEITHER'] = 1.0 - (test['A-coref'] + test['B-coref'])

train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,NEITHER
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,0,Dehner,366,1,http://en.wikipedia.org/wiki/Jeremy_Dehner,0.0
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,1,Alfredo Di St*fano,390,0,http://en.wikipedia.org/wiki/Norberto_Alonso,0.0
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,1,Saddam,295,0,http://en.wikipedia.org/wiki/Aladhadh,0.0
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,0,Pisciotta,536,1,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,0.0
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,1,Rock Reilly,559,0,http://en.wikipedia.org/wiki/Chasers,0.0


Use Bert embeding to acquire correct similarity between A and pronoun, B and pronoun.

In [8]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/extract_features.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2019-05-09 10:45:22--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.01s   

2019-05-09 10:45:23 (2.98 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-05-09 10:45:23--  https://raw.githubusercontent.com/google-research/bert/master/extract_features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13898 (14K) [text/plain]
Saving to: ‘extrac

In [9]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
! 7z x uncased_L-12_H-768_A-12.zip

--2019-05-09 10:45:25--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   127MB/s    in 3.1s    

2019-05-09 10:45:28 (127 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 407727028 bytes (389 MiB)

Extracting archive: uncased_L-12_H-768_A-12.zip
--
Path = uncased_L-12_H-768_A-12.zip
Type = z

In [10]:
import modeling
import extract_features
import tokenization

In [11]:
def compute_offset_no_spaces(text, offset):
    count = 0
    for pos in range(offset):
        if text[pos] != " ": count +=1
    return count

def count_chars_no_special(text):
    count = 0
    special_char_list = ["#"]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count +=1
    return count

def count_length_no_special(text):
    count = 0
    special_char_list = ["#", " "]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count +=1
    return count

In [12]:
def bert_embeddings(df):
    text = df["Text"]
    text.to_csv("input.txt", index = False, header = False)
    
    # run BERT model
    os.system("python3 extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
      --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
      --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
      --layers=-1 \
      --max_seq_length=256 \
      --batch_size=8")
    
    bert_output = pd.read_json("output.jsonl", lines = True)

    os.system("rm output.jsonl")
    os.system("rm input.txt")

    index = df.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"
    
    for i in range(len(df)): # For each line in the data file
        # get the words A, B, Pronoun. Convert them to lower case, since we're using the uncased version of BERT
        P = df.loc[i,"Pronoun"].lower()
        A = df.loc[i,"A"].lower()
        B = df.loc[i,"B"].lower()

        # For each word, find the offset not counting spaces. This is necessary for comparison with the output of BERT
        P_offset = compute_offset_no_spaces(df.loc[i,"Text"], df.loc[i,"Pronoun-offset"])
        A_offset = compute_offset_no_spaces(df.loc[i,"Text"], df.loc[i,"A-offset"])
        B_offset = compute_offset_no_spaces(df.loc[i,"Text"], df.loc[i,"B-offset"])
        # Figure out the length of A, B, not counting spaces or special characters
        A_length = count_length_no_special(A)
        B_length = count_length_no_special(B)

        # Initialize embeddings with zeros
        emb_A = np.zeros(768)
        emb_B = np.zeros(768)
        emb_P = np.zeros(768)

        # Initialize counts
        count_chars = 0
        cnt_A, cnt_B, cnt_P = 0, 0, 0

        features = pd.DataFrame(bert_output.loc[i,"features"])
        
        for j in range(2,len(features)):  # Iterate over the BERT tokens for the current line; we skip over the first 2 tokens, which don't correspond to words
            token = features.loc[j,"token"]

            # See if the character count until the current token matches the offset of any of the 3 target words
            if count_chars  == P_offset: 
                # print(token)
                emb_P += np.array(features.loc[j,"layers"][0]['values'])
                cnt_P += 1
            if count_chars in range(A_offset, A_offset + A_length): 
                # print(token)
                emb_A += np.array(features.loc[j,"layers"][0]['values'])
                cnt_A +=1
            if count_chars in range(B_offset, B_offset + B_length): 
                # print(token)
                emb_B += np.array(features.loc[j,"layers"][0]['values'])
                cnt_B +=1
            # Update the character count
            count_chars += count_length_no_special(token)
        # Taking the average between tokens in the span of A or B, so divide the current value by the count	
        emb_A /= cnt_A
        emb_B /= cnt_B
        
        label = "Neither"
        if (df.loc[i,"A-coref"] == 1):
            label = "A"
        if (df.loc[i,"B-coref"] == 1):
            label = "B"

        # Put everything together in emb
        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
        
    return emb

In [13]:
train_emb = bert_embeddings(train)

In [14]:
test_emb =  bert_embeddings(test)

In [15]:
from scipy import spatial 
def add_similarity_columns(df, df_emb):
    df['sim_A_P'] = 0
    df['sim_B_P'] = 0

    for i in range(0, len(df)):
        sim_A_P = 1 - spatial.distance.cosine(df_emb.loc[i, 'emb_A'], df_emb.loc[i, 'emb_P'])
        if not np.isnan(sim_A_P):
            df.loc[i, 'sim_A_P'] = sim_A_P
        
        sim_B_P = 1 - spatial.distance.cosine(df_emb.loc[i, 'emb_B'], df_emb.loc[i, 'emb_P'])
        if not np.isnan(sim_B_P):
            df.loc[i, 'sim_B_P'] = sim_B_P

In [16]:
add_similarity_columns(train, train_emb)
add_similarity_columns(test, test_emb)

Extend feature matrix

In [17]:
def add_additional_features(df):
    df['Pronoun-offset2'] = df['Pronoun-offset'] + df['Pronoun'].map(len)
    df['A-offset2'] = df['A-offset'] + df['A'].map(len)
    df['B-offset2'] = df['B-offset'] + df['B'].map(len)
    df['A-dist'] = (df['Pronoun-offset'] - df['A-offset']).abs()
    df['B-dist'] = (df['Pronoun-offset'] - df['B-offset']).abs()
    df['section_min'] = df[['Pronoun-offset', 'A-offset', 'B-offset']].min(axis=1)
    df['section_max'] = df[['Pronoun-offset2', 'A-offset2', 'B-offset2']].max(axis=1)
    
add_additional_features(train)
add_additional_features(test)
train.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,NEITHER,sim_A_P,sim_B_P,Pronoun-offset2,A-offset2,B-offset2,A-dist,B-dist,section_min,section_max
ID,,,,,,,,,,,,,,,,,,,,,
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,0,Dehner,366,1,http://en.wikipedia.org/wiki/Jeremy_Dehner,0.0,0.409779,0.389526,386,361,372,31,17,352,386
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,1,Alfredo Di St*fano,390,0,http://en.wikipedia.org/wiki/Norberto_Alonso,0.0,0.385469,0.397535,433,359,408,77,40,353,433
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,1,Saddam,295,0,http://en.wikipedia.org/wiki/Aladhadh,0.0,0.507014,0.322502,314,268,301,56,17,256,314
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,0,Pisciotta,536,1,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,0.0,0.311894,0.348684,529,384,545,149,10,377,545
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,1,Rock Reilly,559,0,http://en.wikipedia.org/wiki/Chasers,0.0,0.436680,0.335950,409,426,570,15,153,406,570


In [18]:
def name_replace(s, r1, r2):
    s = str(s).replace(r1,r2)
    for r3 in r1.split(' '):
        s = str(s).replace(r3,r2)
    return s

train['Text'] = train.apply(lambda r: name_replace(r['Text'], r['A'], 'subjectone'), axis=1)
train['Text'] = train.apply(lambda r: name_replace(r['Text'], r['B'], 'subjecttwo'), axis=1)

test['Text'] = test.apply(lambda r: name_replace(r['Text'], r['A'], 'subjectone'), axis=1)
test['Text'] = test.apply(lambda r: name_replace(r['Text'], r['B'], 'subjecttwo'), axis=1)

In [19]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [20]:
tags = {}
for text in train['Text']:
    doc = nlp(str(text))
    for token in doc:
        if token.text == 'subjectone' or token.text == 'subjecttwo':
            if token.dep_ in tags:
                tags[token.dep_] += 1
            else:
                tags[token.dep_] = 1
                


Find top 5 the most often occurency tags

In [21]:
sorted_tags = sorted(tags.items(), key=lambda kv: kv[1])
sorted_tags[-5:]

[('conj', 676), ('dobj', 722), ('poss', 834), ('pobj', 1523), ('nsubj', 2759)]

In [22]:
def fill_nlp_tag_empty_cols(df, tag):
    df[f"A-{tag}"] = None
    df[f"B-{tag}"] = None
    
def fill_nlp_empty_cols(df, tags):
    for tag in tags:
        fill_nlp_tag_empty_cols(df, tag)

def fill_word_offset_empty_cols(df):
    df['Pronoun-word-offset'] = None
    df['A-word-offset'] = None
    df['B-word-offset'] = None
    df['A-word-dist'] = None
    df['B-word-dist'] = None
    
def fill_similarity(df):
    df['sim_A_P'] = 0.0
    df['sim_B_P'] = 0.0

    
def get_nlp_tag_feature(doc, tag):
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    A_tag = len(tokens[((tokens['text']=='subjectone') & (tokens['dep']==tag))])
    B_tag = len(tokens[((tokens['text']=='subjecttwo') & (tokens['dep']==tag))])
    
    return A_tag, B_tag

def word_offset(doc, w):
    count = 0
    for token in doc:
        if token.text == w:
            break
        if not token.is_punct and token.text != '`':
            count += 1
    return count


def add_nlp_features(df, tags):
    size = len(df)
    fill_nlp_empty_cols(df, tags)
    fill_word_offset_empty_cols(df)
    
    for i in range(0, size):
        text = df.loc[i, 'Text']
        doc = nlp(str(text))
        
        #add tag features
        for tag in tags:
            df.loc[i, f"A-{tag}"], df.loc[i, f"B-{tag}"] = get_nlp_tag_feature(doc, tag)
            
        
        #add word offset features
        df.loc[i, 'Pronoun-word-offset'] = word_offset(doc, df.loc[i, 'Pronoun'])
        df.loc[i, 'A-word-offset'] = word_offset(doc, 'subjectone')
        df.loc[i, 'B-word-offset'] = word_offset(doc, 'subjecttwo')
        
        df.loc[i, 'A-word-dist'] = np.abs(df.loc[i, 'Pronoun-word-offset'] - df.loc[i, 'A-word-offset'])
        df.loc[i, 'B-word-dist'] = np.abs(df.loc[i, 'Pronoun-word-offset'] - df.loc[i, 'B-word-offset'])
        
        #add similarity 
#         df.loc[i, 'sim_A_P'], df.loc[i, 'sim_B_P'] = get_similarity(doc, df.loc[i, 'Pronoun'])


In [23]:
add_nlp_features(train, ['poss', 'nsubj', 'pobj', 'dobj', 'conj'])
add_nlp_features(test, ['poss', 'nsubj', 'pobj', 'dobj', 'conj'])
train

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,NEITHER,sim_A_P,sim_B_P,Pronoun-offset2,A-offset2,B-offset2,A-dist,B-dist,section_min,section_max,A-poss,B-poss,A-nsubj,B-nsubj,A-pobj,B-pobj,A-dobj,B-dobj,A-conj,B-conj,Pronoun-word-offset,A-word-offset,B-word-offset,A-word-dist,B-word-dist
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,0,Dehner,366,1,http://en.wikipedia.org/wiki/Jeremy_Dehner,0.0,0.409779,0.389526,386,361,372,31,17,352,386,0,1,0,1,0,0,0,0,1,0,67,58,8,9,59
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,1,Alfredo Di St*fano,390,0,http://en.wikipedia.org/wiki/Norberto_Alonso,0.0,0.385469,0.397535,433,359,408,77,40,353,433,0,0,1,0,0,1,0,0,0,0,66,56,62,10,4
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,1,Saddam,295,0,http://en.wikipedia.org/wiki/Aladhadh,0.0,0.507014,0.322502,314,268,301,56,17,256,314,0,1,0,0,0,0,0,0,1,0,53,30,50,23,3
3,test-4,"At the trial, subjecttwo said: ``Those who hav...",his,526,Alliata,377,0,Pisciotta,536,1,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,0.0,0.311894,0.348684,529,384,545,149,10,377,545,0,0,0,1,0,1,0,0,2,0,82,17,3,65,79
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,1,Rock Reilly,559,0,http://en.wikipedia.org/wiki/Chasers,0.0,0.436680,0.335950,409,426,570,15,153,406,570,0,0,0,0,0,1,0,0,0,0,50,30,94,20,44
5,test-6,The others were Adam Baldwin (Jayne Cobb in Fi...,her,349,Jewel Staite,281,1,Keller,310,0,http://en.wikipedia.org/wiki/Jennifer_Keller,0.0,0.540487,0.408169,352,293,316,68,39,281,352,0,0,0,0,0,1,1,0,0,0,54,43,33,11,21
6,test-7,"subjectone Fischer (born October 19, 1988) is ...",She,365,Allison,232,1,Grace Smythe,290,0,http://en.wikipedia.org/wiki/Allison_Fischer,0.0,0.569862,0.405694,368,239,302,133,75,232,368,0,0,2,0,0,1,0,0,0,0,56,0,45,56,11
7,test-8,The monster arrives and bites subjecttwo's ton...,her,307,Sophie,248,0,Jeni,277,1,http://en.wikipedia.org/wiki/Leprechaun:_Origins,0.0,0.346924,0.508644,310,254,281,59,30,248,310,0,1,0,0,0,1,0,0,1,0,57,47,5,10,52
8,test-9,"On June 4, 1973 at the Felt Forum, Madison Squ...",he,227,Malave,124,1,Greg Joiner,169,0,http://en.wikipedia.org/wiki/Edwin_Malave,0.0,0.541818,0.521346,229,130,180,103,58,124,229,0,0,1,0,0,1,0,0,0,0,41,13,31,28,10


In [24]:
feature_col = [
               'Pronoun-offset', 
               'Pronoun-offset2', 
               'A-offset', 
               'A-offset2', 
               'A-dist', 
               'B-offset', 
               'B-offset2',
               'B-dist', 
               'section_min',  
               'section_max',
               'A-poss', 
               'B-poss', 
               'A-nsubj',
               'B-nsubj',
               'A-pobj',
               'B-pobj',
               'A-dobj',
               'B-dobj',
               'A-conj',
               'B-conj',
               'sim_A_P',
               'sim_B_P',
               'A-word-offset',
               'B-word-offset',
               'Pronoun-word-offset',
               'A-word-dist',
               'B-word-dist'
              ]
pred_col = ['A-coref', 'B-coref', 'NEITHER']

In [25]:
X_train = train[feature_col].values
Y_train = train[pred_col].values

X_test = test[feature_col].values
Y_test = test[pred_col].values

In [26]:
# x_train, x_test, y_train, y_test = model_selection.train_test_split(train[feature_col].fillna(-1), train[pred_col], test_size=0.2, random_state=1)

In [27]:
model = multiclass.OneVsRestClassifier(ensemble.RandomForestClassifier(max_depth = 7, n_estimators=2000, random_state=33))
model.fit(X_train, Y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=None,
            oob_score=False, random_state=33, verbose=0, warm_start=False),
          n_jobs=None)

In [28]:
model.predict_proba(X_test)

array([[0.34190802, 0.29222693, 0.20937121],
       [0.31247167, 0.538927  , 0.11268243],
       [0.12147312, 0.79797698, 0.05900356],
       ...,
       [0.5090098 , 0.46550054, 0.04556699],
       [0.55475955, 0.27064558, 0.09655326],
       [0.19536943, 0.57158865, 0.1684551 ]])

In [29]:
print('log_loss: ', metrics.log_loss(Y_test, model.predict_proba(X_test)))

log_loss:  0.7050724988663483


In [30]:
# gap_development['A-coref'] = train['A-coref'].astype(int)
# gap_development['B-coref'] = train['B-coref'].astype(int)
# gap_development['NEITHER'] = 1.0 - (train['A-coref'] + train['B-coref'])
# gap_development.head()

In [31]:
# add_additional_features(gap_development)
# gap_development_x = gap_development[feature_col]
# gap_development_y = gap_development[pred_col]
# gap_development_pred = model.predict_proba(gap_development_x)
# print('log_loss: ', metrics.log_loss(gap_development_y, gap_development_pred))

In [32]:
# gap_development_y.head()

In [33]:
# gap_development_pred

Submission test

In [34]:
test_sub1 = pd.read_csv('../input/gendered-pronoun-resolution/test_stage_1.tsv', delimiter='\t')
test_sub1_bert = bert_embeddings(test_sub1)
add_similarity_columns(test_sub1, test_sub1_bert)

#add necessary  features
add_additional_features(test_sub1)

test_sub1['Text'] = test_sub1.apply(lambda r: name_replace(r['Text'], r['A'], 'subjectone'), axis=1)
test_sub1['Text'] = test_sub1.apply(lambda r: name_replace(r['Text'], r['B'], 'subjecttwo'), axis=1)

add_nlp_features(test_sub2, ['poss', 'nsubj', 'pobj', 'dobj', 'conj'])

results = model.predict_proba(test_sub1[feature_col])
test_sub1.rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})
test_sub1['A'] = results[:,0].astype(np.float)
test_sub1['B'] = results[:,1].astype(np.float)
test_sub1['NEITHER'] = results[:,2].astype(np.float)
test_sub1[['ID', 'A', 'B', 'NEITHER']].to_csv('submission1.csv', index=False)
test_sub1.head()

KeyError: 'the label [A-coref] is not in the [index]'

In [35]:
# test_sub2 = pd.read_csv('../input/gendered-pronoun-resolution/test_stage_2.tsv', delimiter='\t')
# test_sub2_bert = bert_embeddings(test_sub2)
# add_similarity_columns(test_sub2, test_sub2_bert)

# #add necessary  features
# add_additional_features(test_sub2)

# test_sub2['Text'] = test_sub2.apply(lambda r: name_replace(r['Text'], r['A'], 'subjectone'), axis=1)
# test_sub2['Text'] = test_sub2.apply(lambda r: name_replace(r['Text'], r['B'], 'subjecttwo'), axis=1)

# add_nlp_features(test_sub2, ['poss', 'nsubj', 'pobj', 'dobj', 'conj'])

# results = model.predict_proba(test_sub2[feature_col])
# test_sub2.rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})
# test_sub2['A'] = results[:,0].astype(np.float)
# test_sub2['B'] = results[:,1].astype(np.float)
# test_sub2['NEITHER'] = results[:,2].astype(np.float)
# test_sub2[['ID', 'A', 'B', 'NEITHER']].to_csv('submission2.csv', index=False)
# test_sub2.head()

In [36]:
test_sub2

NameError: name 'test_sub2' is not defined